# Imports 

In [194]:
import pandas as pd

# Load data 

In [195]:
transactions = pd.read_csv('../data/transactions.csv')

In [196]:
accounts = pd.read_csv('../data/accounts.csv')

In [197]:
users = pd.read_csv('../data/users.csv')

# Explore transactions

In [198]:
transactions.head()

,date,amount,account_id
0,2019-12-16,-29.09,1
1,2019-12-16,160.00,1
2,2019-12-16,-14.00,1
3,2019-12-17,-94.50,1
4,2019-12-17,50.00,1


In [199]:
transactions.shape

(191398, 3)

In [200]:
transactions.describe()

,amount,account_id
count,191398.000000,191398.000000
mean,3.902326,224.494054
std,1202.288020,127.191401
min,-82201.680000,0.000000
25%,-53.700000,110.000000
50%,-15.900000,226.000000
75%,-2.000000,337.000000
max,83340.000000,440.000000


In [201]:
transactions.dtypes

date           object
amount        float64
account_id      int64
dtype: object

In [202]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191398 entries, 0 to 191397
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        191398 non-null  object 
 1   amount      191398 non-null  float64
 2   account_id  191398 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 4.4+ MB


In [203]:
transactions.duplicated().value_counts()

False    181542
True       9856
dtype: int64

In [204]:
transactions.columns

Index(['date', 'amount', 'account_id'], dtype='object')

In [205]:
transactions.date = pd.to_datetime(transactions.date)

In [206]:
transactions.dtypes

date          datetime64[ns]
amount               float64
account_id             int64
dtype: object

# Explore accounts 

In [207]:
accounts.head()

,id,balance,user_id
0,0,3000.00,112
1,1,-465.66,112
2,2,372.81,104
3,3,229.73,198
4,4,1998.48,316


In [208]:
accounts.shape

(441, 3)

In [209]:
accounts.describe()

,id,balance,user_id
count,441.00000,441.000000,441.000000
mean,220.00000,1978.075986,172.231293
std,127.44999,9892.338349,99.099541
min,0.00000,-13474.330000,0.000000
25%,110.00000,0.480000,85.000000
50%,220.00000,59.200000,172.000000
75%,330.00000,1169.360000,258.000000
max,440.00000,154251.970000,342.000000


In [210]:
accounts.dtypes

id           int64
balance    float64
user_id      int64
dtype: object

In [211]:
accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       441 non-null    int64  
 1   balance  441 non-null    float64
 2   user_id  441 non-null    int64  
dtypes: float64(1), int64(2)
memory usage: 10.5 KB


In [212]:
accounts.duplicated().value_counts()

False    441
dtype: int64

In [213]:
accounts.columns

Index(['id', 'balance', 'user_id'], dtype='object')

In [214]:
accounts.user_id.nunique()

343

In [215]:
accounts.user_id[accounts.user_id.duplicated()]

1      112
13     197
18     243
21     146
23     222
      ... 
426    156
427    156
429     13
436    326
440     77
Name: user_id, Length: 98, dtype: int64

In [216]:
accounts[accounts.user_id == 156]

,id,balance,user_id
424,424,230.50,156
425,425,5.14,156
426,426,1.15,156
427,427,234.87,156


# Explore users

In [218]:
users.head()

,id;update_date
0,0;27/09/2020
1,1;27/09/2020
2,2;27/09/2020
3,3;27/09/2020
4,4;27/09/2020


In [219]:
users.shape

(343, 1)

In [220]:
users.columns

Index(['id;update_date'], dtype='object')

In [221]:
# users has one column separated with colon, let's make it 2 columns
# rename id to user_id to further merge with other datasets
users[['user_id', 'update_date']] = users['id;update_date'].str.split(";",expand=True,)
# drop splitted column
users.drop(columns='id;update_date',inplace=True)


# or could have loaded from csv separeated with  colon ->
# users = pd.read_csv('../data/users.csv', sep=';')

In [222]:
users.describe()

,user_id,update_date
count,343,343
unique,343,2
top,0,27/09/2020
freq,1,312


In [223]:
users.head()

,user_id,update_date
0,0,27/09/2020
1,1,27/09/2020
2,2,27/09/2020
3,3,27/09/2020
4,4,27/09/2020


In [224]:
users.dtypes

user_id        object
update_date    object
dtype: object

In [225]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      343 non-null    object
 1   update_date  343 non-null    object
dtypes: object(2)
memory usage: 5.5+ KB


In [226]:
users.duplicated().value_counts()

False    343
dtype: int64

In [227]:
users.columns

Index(['user_id', 'update_date'], dtype='object')

In [228]:
users.update_date = pd.to_datetime(users.update_date)

In [229]:
users.user_id = users.user_id.astype('int')

In [230]:
users.dtypes

user_id                 int64
update_date    datetime64[ns]
dtype: object

# Merge datasets

In [189]:
transactions.columns

Index(['date', 'amount', 'account_id'], dtype='object')

In [164]:
accounts.columns

Index(['account_id', 'balance', 'user_id'], dtype='object')

In [165]:
users.columns

Index(['user_id', 'update_date'], dtype='object')

In [167]:
# the balance provided for each account is the balance at the update date of the user the account belongs to.
# so merge users and accounts
users_balance = users.merge(accounts,on='user_id')

In [166]:
users_balance.columns


Index(['user_id', 'update_date', 'account_id', 'balance'], dtype='object')

In [176]:
users_balance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441 entries, 0 to 440
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      441 non-null    int64         
 1   update_date  441 non-null    datetime64[ns]
 2   account_id   441 non-null    int64         
 3   balance      441 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 17.2 KB


In [140]:
# rename columns title for further merge
accounts.rename(columns={'id':'account_id'},inplace=True)

In [ ]:
users_balance.rename(columns={'update_date':'date'},inplace=True)

In [234]:
users_balance

,user_id,date,account_id,balance
0,0,2020-09-27,39,0.77
1,0,2020-09-27,40,2.16
2,1,2020-09-27,305,2514.30
3,1,2020-09-27,306,1336.54
4,2,2020-09-27,161,88.23
...,...,...,...,...
436,339,2020-09-26,257,2834.90
437,340,2020-09-27,52,-165.20
438,340,2020-09-27,53,1.53
439,341,2020-09-27,11,21.44


In [240]:
transactions[transactions.account_id==53]

,date,amount,account_id
23387,2020-08-17,-75.66,53
23388,2020-08-17,-105.09,53
23389,2020-08-17,-4.00,53
23390,2020-08-17,200.00,53
23391,2020-08-20,-10.99,53
23392,2020-08-22,-3.80,53
23393,2020-09-01,-0.99,53
23394,2020-09-04,76.95,53
23395,2020-09-04,-9.99,53
23396,2020-09-04,-59.99,53


In [142]:
# merge transaction and users_balance
#merged_df = transactions.merge(accounts, on = 'account_id').merge(users, on='user_id')

# Explore and clean merged df 

In [143]:
merged_df.head()

,date,amount,account_id,balance,user_id,update_date
0,2019-12-16,-29.09,1,-465.66,112,2020-09-26
1,2019-12-16,160.00,1,-465.66,112,2020-09-26
2,2019-12-16,-14.00,1,-465.66,112,2020-09-26
3,2019-12-17,-94.50,1,-465.66,112,2020-09-26
4,2019-12-17,50.00,1,-465.66,112,2020-09-26


In [144]:
merged_df.duplicated().value_counts()

False    181542
True       9856
dtype: int64

In [145]:
merged_df.drop_duplicates(inplace=True)

In [148]:
merged_df.shape

(181542, 6)

In [150]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181542 entries, 0 to 191397
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         181542 non-null  datetime64[ns]
 1   amount       181542 non-null  float64       
 2   account_id   181542 non-null  int64         
 3   balance      181542 non-null  float64       
 4   user_id      181542 non-null  int64         
 5   update_date  181542 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2)
memory usage: 9.7 MB


In [152]:
merged_df.describe()
# numerical data in dataset don't contain anything but numbers, 
#as it is expected by its column names, 
# i.e., no weird values or caracters or nulls, etc. 
#can start working with it


,amount,account_id,balance,user_id
count,181542.000000,181542.000000,181542.000000,181542.000000
mean,4.840099,225.050369,3289.855208,157.207649
std,1230.856862,127.042141,15299.718685,100.493586
min,-82201.680000,0.000000,-13474.330000,0.000000
25%,-56.500000,110.000000,1.400000,65.000000
50%,-16.750000,227.000000,220.300000,150.000000
75%,-2.340000,337.000000,1474.320000,250.000000
max,83340.000000,440.000000,154251.970000,342.000000


In [154]:
# Build a function to check which accounts have more than 180 days of history


In [ ]:
# You can assume that any account passed to your service will have at least 6 months of history.